In [2]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices,char_to_idx,idx_to_char,vocab_size)=d2l.load_data_jay_lyrics()


In [3]:
#产生bat_size个词的向量
def one_hot(x,n_class,dtype=torch.float32):
    x=x.long()
    res=torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1,x.view(-1,1),1)
    return res

x=torch.tensor([0,2])
one_hot(x,vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [5]:
#比如（2，5） 即产生5个batch_size个词的列表
def to_onehot(X,n_class):
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])]
#=
X=torch.arange(10).view(2,5)
inputs=to_onehot(X,vocab_size)
print(len(inputs),inputs[0].shape)

5 torch.Size([2, 1027])


In [18]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
print('will use',device)

def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    
    W_xh=_one((num_inputs,num_hiddens))
    W_hh=_one((num_hiddens,num_hiddens))
    b_h=torch.nn.Parameter(torch.zeros(num_hiddens,device=device,requires_grad=True))
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device,requires_grad=True))
    return nn.ParameterList([W_xh,W_hh,b_h,W_hq,b_q])


will use cuda


In [19]:
def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)

In [22]:
def rnn(inputs,state,params):
    W_xh,W_hh,b_h,W_hq,b_q=params
    H, =state  
    outputs=[]
    for X in inputs:
        H=torch.tanh(torch.matmul(X,W_xh)+torch.matmul(H,W_hh)+b_h)
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

In [24]:
state=init_rnn_state(X.shape[0],num_hiddens,device)
inputs=to_onehot(X.to(device),vocab_size)
params=get_params()
outputs,state_new=rnn(inputs,state,params)
print(len(outputs),outputs[0].shape,state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


In [34]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,
               num_hiddens,vocab_size,device,idx_to_char,char_to_idx):
    state=init_rnn_state(1,num_hiddens,device)
    output=[char_to_idx[prefix[0]]]  #首个字符对应的索引
    for t in range(num_chars+len(prefix)-1):
        X=to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        (Y,state)=rnn(X,state,params)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [35]:
predict_rnn('分开',10,rnn,params,init_rnn_state,num_hiddens,vocab_size,
           device,idx_to_char,char_to_idx)

'分开腿等染狠囱躺怎家边小'

In [36]:
#假如梯度本身就很小了再缩小不是更会出现梯度衰减的情况么??
def grad_clipping(params,theta,device):
    norm=torch.tensor([0.0],device=device)
    for param in params:
        norm+=(param.grad.data**2).sum()
    norm=norm.sqrt().item()
    if norm>theta:
        for param in params:
            param.grad.data*=(theta/norm)

In [61]:
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,
                         vocab_size,device,corpus_indices,idx_to_char,
                         char_to_idx,is_random_iter,num_epochs,num_steps,
                         lr,clipping_theta,batch_size,pred_period,pred_len,prefixes):
    if is_random_iter:
        data_iter_fn=d2l.data_iter_random
    else:
        data_iter_fn=d2l.data_iter_consecutive
    params=get_params()
    loss=nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:
            state=init_rnn_state(batch_size,num_hiddens,device)
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_fn(corpus_indices,batch_size,num_steps,device)
        for X,Y in data_iter:
            if is_random_iter:
                state=init_rnn_state(batch_size,num_hiddens,device)
            else:
                for s in state:
                    s.detach_()
                    
            inputs=to_onehot(X,vocab_size)
            (outputs,state)=rnn(inputs,state,params)
            outputs=torch.cat(outputs,dim=0)
            y=torch.transpose(Y,0,1).contiguous().view(-1)
            l=loss(outputs,y.long())
            
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params,clipping_theta,device)
            d2l.sgd(params,lr,1)
            l_sum+=l.item()*y.shape[0]   
            n+=y.shape[0] ##因为我这里要求的是所有epoch累计的损失函数的平均值
            
        if (epoch+1) % pred_period==0:
            print('epoch %d,perplexity %f,time %.2f sec'%
                   (epoch+1,math.exp(l_sum/n),time.time()-start))
            for prefix in prefixes:
                print(' -',predict_rnn(prefix,pred_len,rnn,params,
                                       init_rnn_state,num_hiddens,vocab_size,
                                     device,idx_to_char,char_to_idx))
                

In [62]:
num_epochs,num_steps,batch_size,lr,clipping_theta=250,35,32,1e2,1e-2
pred_period,pred_len,prefixes=50,50,['分开','不分开']
train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,
                     vocab_size,device,corpus_indices,idx_to_char,
                     char_to_idx,True,num_epochs,num_steps,lr,clipping_theta,batch_size,
                     pred_period,pred_len,prefixes)

epoch 50,perplexity 68.019595,time 0.55 sec
 - 分开 我想要这生  不知我 你子我 别子就 我想就的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的
 - 不分开 我想要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
epoch 100,perplexity 9.808334,time 0.58 sec
 - 分开 有时的让我面红的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱
 - 不分开亚 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离生我 不知不觉 我已经这节奏 后知后觉 
epoch 150,perplexity 2.900928,time 0.59 sec
 - 分开 有子我不抽功是 但杰个人已经不是我 没有你在 我有了烦力向 世果我遇见你是一场悲剧 我可以让生命就
 - 不分开简 我叫你烦 你打我妈 这样了看不屈 一身正气 快使用双截棍 哼哼哈兮 快使用人截棍 哼哼哈兮 快果
epoch 200,perplexity 1.592572,time 0.61 sec
 - 分开 一只在停留仪七等我年愿的生 伤血呢木的玄武 双什激动 一颗心到现在还在抽痛 还为分手前那句抱歉 草
 - 不分开期 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱
epoch 250,perplexity 1.310057,time 0.55 sec
 - 分开不会再藤 一切画最 不血跳动 染红夜蒙 过去种种 你一场痛 不再不动 你都梦  不的梦空 不句变发 
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 刻点酸枪 你一放空 一再不通 你
